# Harvest System Mix Explorer

Analyse machine-role allocation and system mixes for the synthetic medium scenario.

In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve()
while PROJECT_ROOT != PROJECT_ROOT.parent and not (PROJECT_ROOT / "pyproject.toml").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent
if not (PROJECT_ROOT / "pyproject.toml").exists():
    raise RuntimeError(
        "Notebook must be executed within a FHOPS checkout (pyproject.toml not found)."
    )
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))


import pandas as pd
import yaml

from fhops.scenario.io import load_scenario
from fhops.scenario.synthetic import SyntheticDatasetConfig, generate_random_dataset

SCENARIO = PROJECT_ROOT / "examples/synthetic/medium/scenario.yaml"
ASSIGNMENTS = PROJECT_ROOT / "docs/examples/analytics/data/synthetic_medium_sa_assignments.csv"

## System Mix Metadata

In [2]:
metadata = yaml.safe_load((PROJECT_ROOT / "examples/synthetic/metadata.yaml").read_text())
medium_meta = metadata["medium"]
medium_meta.get("system_mix", {})

{}

## Machine Roles in Scenario

In [3]:
scenario = load_scenario(SCENARIO)
roles = pd.Series([machine.role or "unassigned" for machine in scenario.machines], name="role")
roles.value_counts()

role
harvester    2
forwarder    2
Name: count, dtype: int64

## Generate Custom Mix

In [4]:
custom_config = SyntheticDatasetConfig(
    name="synthetic-mix-demo",
    tier="large",
    num_blocks=12,
    num_days=16,
    num_machines=6,
    system_mix={"ground_fb_skid": 0.5, "ctl": 0.3, "steep_tethered": 0.2},
)
bundle = generate_random_dataset(custom_config, seed=404)
custom_roles = bundle.machines["role"].fillna("unassigned")
custom_roles.value_counts()

role
harvester    3
forwarder    3
Name: count, dtype: int64

## Role Comparison

In [5]:
comparison_df = pd.DataFrame(
    {"scenario_roles": roles.value_counts(), "custom_mix_roles": custom_roles.value_counts()}
).fillna(0)
comparison_df

,scenario_roles,custom_mix_roles
role,,
harvester,2,3
forwarder,2,3
